In [ ]:
!pip install opencv-python
!pip install mediapipe

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.6/33.6 MB 39.8 MB/s eta 0:00:00


In [ ]:

import mediapipe as mp
# import dependencies
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)

  # get photo data
  data = eval_js('takePhoto({})'.format(quality))
  # get OpenCV format image
  # img = 
  img = js_to_image(data) 
  
  cv2.imwrite(filename, img)

  return filename

In [ ]:
try:
  filename = take_photo('photo.jpg')
  print('Saved to {}'.format(filename))
  
  # Show the image which was just taken.
  img = Image(filename)
  display(img)
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

<IPython.core.display.Javascript object>

name 'js_to_image' is not defined


In [ ]:
#import cv2
#import mediapipe as mp
#import time
#from realsense_depth import *

#cam = DepthCamera()
#cam = cv2.VideoCapture(0)


mpHands = mp.solutions.hands #cria um objeto do media pipe com as marcações da mão
hands =mpHands.Hands() 
mpDraw = mp.solutions.drawing_utils 

while True: #loop
    #ret, d_frame, img = cam.get_frame()
    #ret,  img = cam.read()
    filename = take_photo('photo.jpg')
    imgRGB = cv2.cvtColor(Image(filename),cv2.COLOR_BGR2RGB) #convertendo a imagem de BGR para RGB
    resultado = hands.process(imgRGB) #faz o processamento das informações do objeto tipo mão
    handPoints = resultado.multi_hand_landmarks  #leitura dos resultados do processamento
    h,w,_ = img.shape #pegando dimensões da imagem
    pontos = []

    if resultado.multi_hand_landmarks:
        for handsLms in resultado.multi_hand_landmarks:
            mpDraw.draw_landmarks(img, handsLms, mpHands.HAND_CONNECTIONS) #desenha os pontos da mão e as conexões entre eles 
            for id,cord in enumerate(handsLms.landmark): #enumera as marcações da mão e obtem e armazena as coordenadas de cada uma dentro de um vetor
                cx,cy = int(cord.x * w),int(cord.y * h)
                pontos.append((cx,cy))

            dedos = [8,12,16,20]
            cont = 0
            if pontos: #lê qual número está mostrado na mão a partir dos "intervalos dos dedos" mostrados nas marcações
                if pontos[4][0] < pontos[3][0]:
                    cont += 1
                for x in dedos:
                   if pontos[x][1] < pontos[x-2][1]:
                       cont +=1

            cv2.rectangle(img, (80, 10), (200,110), (255, 0, 0), -1) #mostrando número lido na tela
            cv2.putText(img,str(cont),(100,100),cv2.FONT_HERSHEY_SIMPLEX,4,(255,255,255),5) 
            #print(contador)
    



    cv2.imshow("imagem",img)
    key =cv2.waitKey(1)
    if key == 27:#
        break

    

<IPython.core.display.Javascript object>

NameError: ignored